In [1]:
import cv2
from tensorflow import keras
import tensorflow as tf
from PIL import Image
import numpy as np


In [2]:
#Load data stuff and model
labels_names =['Deer','Human','Unknown']
data_train = np.load('data.npy')
labels_train = np.load('labels.npy')
labels_train = labels_train.astype(int)
class_labels, _ = np.unique(labels_train, return_counts=True)

model = keras.models.load_model('retrained_model_ResNet50_1.h5')

In [4]:
#Example with random deer image
image_path = 'test_validation_images/deer_3.png'
image = Image.open(image_path).convert('RGB')


image_array = np.array(image)

img_height = 224
img_width = 224

img = tf.image.resize(image_array, size=[img_height,img_width])
img =tf.expand_dims(img,0)

# Predictions
predictions = model.predict(img)
score = tf.nn.softmax(predictions[0])

print(
    "Class: {} - {} | {:.2f} %"
    .format(class_labels[np.argmax(score)], labels_names[class_labels[np.argmax(score)]-1] , 100 * np.max(score))
)


1/1 [==============================] - 1s 992ms/step
Class: 1 - Deer | 100.00 %


In [ ]:

def capture_image():
    camera = cv2.VideoCapture(0)  

    while True:
        ret, frame = camera.read()
        cv2.imshow("Camera", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

    camera.release()
    cv2.destroyAllWindows()


    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    return image


captured_image = capture_image()

resized_image = captured_image.resize((img_width, img_height))
image_array = np.array(resized_image)
img = tf.expand_dims(image_array, 0)


predictions = model.predict(img)
score = tf.nn.softmax(predictions[0])

predicted_class = np.argmax(score)
class_label = labels_names[predicted_class]


print(
    "Class: {} - {} | {:.2f} %"
    .format(class_labels[np.argmax(score)], labels_names[class_labels[np.argmax(score)]-1] , 100 * np.max(score))
)
image = cv2.imread("contour_detector.png")

cv2.imshow("yo",gray)

1/1 [==============================] - 1s 615ms/step
Class: 2 - Human | 100.00 %


In [17]:
def capture_image():
    camera = cv2.VideoCapture(0)  

    while True:
        ret, frame = camera.read()
        cv2.imshow("Camera", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

    camera.release()
    cv2.destroyAllWindows()

    return frame


image = cv2.imread("contour_detector.png")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred = cv2.GaussianBlur(gray, (5, 5), 0)

edged = cv2.Canny(blurred, 75, 200)


cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]

screenCnt = None

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.05 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        print("success")
        break

if screenCnt is not None:
    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)

    cv2.imshow("Outline", image)
    print("2x success")
else:
    print("No paper-like contour found.")

cv2.waitKey(0)
cv2.destroyAllWindows()